In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy.stats import pearsonr
from tqdm.notebook import tqdm

In [2]:
## file path
input_data_dir = '/global/cfs/cdirs/kbase/KE-Catboost/ziming/taxonomy/data'
input_file = os.path.join(input_data_dir, 'taxonomy_v4.1','feature_transpose.tsv')
output_file = os.path.join(input_data_dir, 'taxonomy_v4.1', 'feature_correlation.tsv')

In [3]:
## ~4 min, 23GB
## read input feature file 
X = []
feature_name = []
input_file_row_count = 20059
with open(input_file, 'r') as f_in:
    reader = csv.reader(f_in, delimiter="\t")
    header = next(reader)
    print(header[:10]) 
    for i in tqdm(range(input_file_row_count)):
        row = next(reader)
        feature_name.append(row[0])
        X.append([float(_) for _ in row[1:]]) # skip feature name and convert str to float

['feature', '0', '1', '2', '3', '4', '5', '6', '7', '8']


In [4]:
%%time
## Wall time: 46 s
X = np.array(X)

CPU times: user 40.8 s, sys: 5.58 s, total: 46.3 s
Wall time: 46 s


In [5]:
def pearson_correlation_coefficient(X):
    """
    calulate pearson correlation coefficient of matrix X
    X: numpy array (MxN)
    return pcc: numpy array (MxM)
    """
    M, N = X.shape[0], X.shape[1] # number of features, number of datapoints
    X_mean = np.mean(X, axis = 1).reshape(X.shape[0], 1)
    X_std = np.std(X, axis = 1).reshape(X.shape[0], 1)
    X_tilde = (X-X_mean)/X_std
    pcc = X_tilde@X_tilde.T/N
    return pcc

In [6]:
%%time
pcc_X = pearson_correlation_coefficient(X)
print(pcc_X.shape)
pcc_X

(20059, 20059)
CPU times: user 5min 51s, sys: 19.8 s, total: 6min 10s
Wall time: 3min 14s


array([[ 1.00000000e+00, -1.17123797e-04, -3.40458615e-04, ...,
        -1.17123797e-04, -4.68610953e-04,  5.42070167e-02],
       [-1.17123797e-04,  1.00000000e+00,  1.18006963e-03, ...,
        -3.29348220e-05, -1.31771841e-04, -1.34768289e-04],
       [-3.40458615e-04,  1.18006963e-03,  1.00000000e+00, ...,
        -9.57358298e-05, -3.83037946e-04, -3.91748101e-04],
       ...,
       [-1.17123797e-04, -3.29348220e-05, -9.57358298e-05, ...,
         1.00000000e+00, -1.31771841e-04, -1.34768289e-04],
       [-4.68610953e-04, -1.31771841e-04, -3.83037946e-04, ...,
        -1.31771841e-04,  1.00000000e+00, -5.39206362e-04],
       [ 5.42070167e-02, -1.34768289e-04, -3.91748101e-04, ...,
        -1.34768289e-04, -5.39206362e-04,  1.00000000e+00]])

In [7]:
def test_pearson_correlation_coefficient():
    """
    test pearson_correlation_coefficient against scipy.stats pearsonr method
    """
    ## randomly generate a 3x4 array
    test_array = np.random.rand(3,4)
    ## my pcc value matrix
    my_pcc = pearson_correlation_coefficient(test_array)
    ## initilize ground truth pcc matrix
    true_pcc = np.ones((test_array.shape[0],test_array.shape[0]))
    ## update value by scipy.stats pearsonr method
    for i in range(test_array.shape[0]):
        for j in range(test_array.shape[0]):
            if i != j:
                p, v = pearsonr(test_array[i], test_array[j])
                true_pcc[i][j] = p
                true_pcc[j][i] = p
    
    assert (np.round(my_pcc, 5) == np.round(true_pcc, 5)).all()
    print("OK!")
    
    
test_pearson_correlation_coefficient()   

OK!


In [9]:
## write pcc matrix to a new file
with open(output_file, 'w') as f_out:
    writer = csv.writer(f_out, delimiter="\t")
    writer.writerow([None] + feature_name)
    for i in tqdm(range(len(feature_name))):
        writer.writerow([feature_name[i]]+list(pcc_X[i]))
print("Done!")


Done!
